In [1]:
import os
from dotenv import load_dotenv
from pymongo import MongoClient

load_dotenv()

# MongoDB connection
MONGO_URI = os.getenv("MONGO_URI")
client = MongoClient(MONGO_URI)
db = client["TaskAssistant"]
tasks_collection = db["tasks"]

tasks_collection.insert_one({"name": "Hadil", "email": "example@gmail.com"})



InsertOneResult(ObjectId('69178acf3d880d306d48b83c'), acknowledged=True)

In [ ]:
import subprocess
import sys
import os
import json

# Test if server starts
server_path = "backend/server.py"  # Adjust this to your actual path
print(f"Testing server at: {"backend\\server.py"}")

try:
    # Start the server process
    process = subprocess.Popen(
        [sys.executable, server_path],
        stdin=subprocess.PIPE,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True
    )
    
    # Send an MCP initialize request
    init_request = {
        "jsonrpc": "2.0",
        "id": 1,
        "method": "initialize",
        "params": {
            "protocolVersion": "2024-11-05",
            "capabilities": {},
            "clientInfo": {
                "name": "test-client",
                "version": "1.0.0"
            }
        }
    }
    
    # Write request
    process.stdin.write(json.dumps(init_request) + "\n")
    process.stdin.flush()
    
    # Try to read response (with timeout)
    import select
    import time
    
    # Wait a bit for response
    time.sleep(2)
    
    # Check if process is still running
    if process.poll() is not None:
        # Process terminated
        stdout, stderr = process.communicate()
        print("❌ Server terminated immediately!")
        print(f"\nSTDOUT:\n{stdout}")
        print(f"\nSTDERR:\n{stderr}")
        print(f"\nReturn code: {process.returncode}")
    else:
        # Try to read output
        output = process.stdout.readline()
        print(f"✅ Server responded: {output}")
        process.terminate()
        
except Exception as e:
    print(f"❌ Error testing server: {e}")
    import traceback
    traceback.print_exc()

NameError: name '__file__' is not defined